Reconocimiento de texto
===

* *30 min* | Ultima modificación: Junio 22, 2019

## Definición del problema

En los problemas de OCR (optical character recognition), el reconocimiento se realizar mediante la división del documento en una malla tal que cada caracter es aislado en una celda. El problema real consiste en identificar el caracter aislado. La dificultad radica que un mismo caracter puede variar en tamaño, forma de escritura, grosor y orientación; véase la figura de abajo. Se desea construir una herramienta que permita el reconocimiento automático de caracteres.  



<img src="images/img00.png" height=500 width=300>

Se tiene una base de 20.000 ejemplos de 16 atributos, la cual fue construida con las 26 letras mayúsculas del alfabeto ingles, impresas en blanco y negro distorsionadas y escaladas. Los atributos son los siguientes:

1.	lettr	capital letter	(26 values from A to Z) 
2.	x-box	horizontal position of box	(integer) 
3.	y-box	vertical position of box	(integer) 
4.	width	width of box	(integer) 
5.	high height of box	(integer) 
6.	onpix	total # on pixels	(integer) 
7.	x-bar	mean x of on pixels in box	(integer) 
8.	y-bar	mean y of on pixels in box	(integer) 
9.	x2bar	mean x variance	(integer) 
10.	y2bar	mean y variance	(integer) 
11.	xybar	mean x y correlation	(integer) 
12.	x2ybr	mean of x * x * y	(integer) 
13.	xy2br	mean of x * y * y	(integer) 
14.	x-ege	mean edge count left to right	(integer) 
15.	xegvy	correlation of x-ege with y	(integer) 
16.	y-ege	mean edge count bottom to top	(integer) 
17.	yegvx	correlation of y-ege with x	(integer)


El problema en términos de los datos consiste en poder asociar el patrón capturado en cada celda con la correspondiente letra.  



## Exploración

In [4]:
## Carga de los datos
data <- read.csv("data/letterdata.csv")
str(data)

'data.frame':	20000 obs. of  17 variables:
 $ letter: Factor w/ 26 levels "A","B","C","D",..: 20 9 4 14 7 19 2 1 10 13 ...
 $ xbox  : int  2 5 4 7 2 4 4 1 2 11 ...
 $ ybox  : int  8 12 11 11 1 11 2 1 2 15 ...
 $ width : int  3 3 6 6 3 5 5 3 4 13 ...
 $ height: int  5 7 8 6 1 8 4 2 4 9 ...
 $ onpix : int  1 2 6 3 1 3 4 1 2 7 ...
 $ xbar  : int  8 10 10 5 8 8 8 8 10 13 ...
 $ ybar  : int  13 5 6 9 6 8 7 2 6 2 ...
 $ x2bar : int  0 5 2 4 6 6 6 2 2 6 ...
 $ y2bar : int  6 4 6 6 6 9 6 2 6 2 ...
 $ xybar : int  6 13 10 4 6 5 7 8 12 12 ...
 $ x2ybar: int  10 3 3 4 5 6 6 2 4 1 ...
 $ xy2bar: int  8 9 7 10 9 6 6 8 8 9 ...
 $ xedge : int  0 2 3 6 1 0 2 1 1 8 ...
 $ xedgey: int  8 8 7 10 7 8 8 6 6 1 ...
 $ yedge : int  0 4 3 2 5 9 7 2 1 1 ...
 $ yedgex: int  8 10 9 8 10 7 10 7 7 8 ...


## Entrenamiento del modelo

In [24]:
##
## Se usa el 80% de los datos para el entrenamiento 
## y el 20% restante para la prueba.
##
data_train <- data[1:16000,]
data_test  <- data[16001:20000,]

In [25]:
## install.packages("kernlab")
library(kernlab)

In [26]:
clf <- ksvm(letter ~ ., 
            data = data,
           kernel="vanilladot")

 Setting default kernel parameters  


**Ejercicio.--** Consulte la ayuda de la función ksvm y explique los tipos de funciones de kernel disponibles.

## Evaluación del modelo

In [27]:
##
## Se pronóstica la letra correspondiente 
## para la muestra de prueba
##
y_test_pred <- predict(clf, data_test)
head(y_test_pred)

[1] U N V I N H
Levels: A B C D E F G H I J K L M N O P Q R S T U V W X Y Z

In [28]:
##
## La siguiente tabla indica la letra real y
## la letra reconocida. Las columnas corresponden
## a la letra real, mientras que las filas corresponden
## a las letras reconocidas.
##
table(y_test_pred, data_test$letter)

           
y_test_pred   A   B   C   D   E   F   G   H   I   J   K   L   M   N   O   P   Q
          A 145   0   0   0   0   0   0   0   0   1   0   0   0   0   2   0   6
          B   0 122   0   5   2   0   1   1   0   0   1   0   2   0   0   2   2
          C   0   0 121   0   1   0   9   2   1   0   1   3   0   0   2   0   0
          D   1   1   0 156   0   0   2   9   4   2   2   2   0   3   6   3   1
          E   0   0   6   0 129   2   1   0   0   0   3   4   0   0   0   0   3
          F   0   0   0   0   0 138   1   3   5   0   0   0   0   1   0  15   0
          G   1   1   3   2   9   2 126   3   0   0   0   2   1   0   0   2   8
          H   0   0   0   2   0   2   0 108   0   2   4   2   2   5  17   0   1
          I   0   0   0   0   0   0   0   0 144   7   0   0   0   0   0   0   0
          J   0   1   0   0   0   1   0   1   4 130   0   0   0   0   0   1   2
          K   1   1   8   0   0   0   3   6   0   0 122   0   0   2   0   1   0
          L   0   0   0   0 

In [29]:
##
## número de casos correctos e incorrectos
##
agreement <- y_test_pred == data_test$letter
table(agreement)

agreement
FALSE  TRUE 
  570  3430 

In [30]:
prop.table(table(agreement))

agreement
 FALSE   TRUE 
0.1425 0.8575 

## Mejora del modelo

In [32]:
clf_rbf <- ksvm(letter ~ ., 
                data = data_train,
                kernel = "rbfdot")

y_test_pred_rbf <- predict(clf_rbf,
                           data_test)

agreement_rbf <- y_test_pred_rbf == y_test_true
table(agreement_rbf)

agreement_rbf
FALSE  TRUE 
  277  3723 

In [33]:
prop.table(table(agreement_rbf))

agreement_rbf
  FALSE    TRUE 
0.06925 0.93075 

**Ejercicio.--** Realice la estimación usando cross-validation.